<a href="https://colab.research.google.com/github/ujjalkumarmaity/NLP/blob/main/Huggingface-NLP-Course/5-datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets

## Working with local and remote datasets

|Data format  |	Loading script |  Example
|-- | -- | -- |
| CSV & TSV	| csv |	load_dataset("csv", data_files="my_file.csv") |
| Text files	| text	| load_dataset("text", data_files="my_file.txt") |
| JSON & JSON Lines	| json |	load_dataset("json", data_files="my_file.jsonl") |
| Pickled DataFrames |	pandas	| load_dataset("pandas", data_files="my_dataframe.pkl") |


In [ ]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

--2024-01-26 11:11:48--  https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz [following]
--2024-01-26 11:11:48--  https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7725286 (7.4M) [application/octet-stream]
Saving to: ‘SQuAD_it-train.json.gz’

SQuAD_it-train.json 100%[===================>]   7.37M  --.-KB/s    in 0.09s   

2024-01-26 11:11:48 (83.2 MB/s) - ‘SQuAD_it-train.json.gz’ saved [7725286/7725286]

--202

In [ ]:
!gzip -dkv SQuAD_it-*.json.gz

SQuAD_it-test.json.gz:	 87.5% -- created SQuAD_it-test.json
SQuAD_it-train.json.gz:	 82.3% -- created SQuAD_it-train.json


In [ ]:
from datasets import load_dataset
data = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['paragraphs', 'title'],
        num_rows: 442
    })
})

## slice and dice data

In [ ]:
from datasets import load_dataset
data = load_dataset("glue", "mrpc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
train_sample = data['train'].select(range(10))
train_sample

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 10
})

In [ ]:
train_sample[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

### Lower case conversion

In [ ]:
def lower_case(x):
    return {'sentence1':x['sentence1'].lower()}
train_sample = train_sample.map(lower_case)
train_sample[0]['sentence1']

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

'amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'

### Filter

In [ ]:
train_sample.filter(lambda x:len(x['sentence1'])>50)

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 10
})

### Creating new columns

In [ ]:
def lower_case(x):
    return {'len_sentence1':len(x['sentence1'])}
train_sample = train_sample.map(lower_case)
train_sample

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'len_sentence1'],
    num_rows: 10
})

### Map

### From Dataset s to DataFrame s and back

In [ ]:
from datasets import load_dataset
data = load_dataset("glue", "mrpc")
data = data['train']

In [ ]:
data.set_format("pandas")

In [ ]:
data[:3]

,sentence1,sentence2,label,idx
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ...",1,2


In [ ]:
type(data)

datasets.arrow_dataset.Dataset

In [ ]:
df = data[:]
type(df)

pandas.core.frame.DataFrame

In [ ]:
df.head(2)

,sentence1,sentence2,label,idx
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1


In [ ]:
from datasets import Dataset
data_ = Dataset.from_pandas(df)

In [ ]:
data_

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

## Semantic search with FAISS

In [ ]:
from datasets import load_dataset
dataset = load_dataset("lewtun/github-issues", split="train")
dataset

In [ ]:
remove_columns = list(set(dataset.column_names) - set(['html_url', 'title', 'comments', 'body']))
dataset = dataset.remove_columns(remove_columns)

In [ ]:
dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 3019
})

In [ ]:
dataset[:2]

{'html_url': ['https://github.com/huggingface/datasets/pull/2955',
  'https://github.com/huggingface/datasets/pull/2954'],
 'title': ['Update legacy Python image for CI tests in Linux',
  'Run tests in parallel'],
 'comments': [[],
  ['There is a speed up in Windows machines:\r\n- From `13m 52s` to `11m 10s`\r\n\r\nIn Linux machines, some workers crash with error message:\r\n```\r\nOSError: [Errno 12] Cannot allocate memory\r\n```',
   'There is also a speed up in Linux machines:\r\n- From `7m 30s` to `5m 32s`']],
 'body': ['Instead of legacy, use next-generation convenience images, built from the ground up with CI, efficiency, and determinism in mind. Here are some of the highlights:\r\n\r\n- Faster spin-up time - In Docker terminology, these next-gen images will generally have fewer and smaller layers. Using these new images will lead to faster image downloads when a build starts, and a higher likelihood that the image is already cached on the host.\r\n\r\n- Improved reliability and 

In [ ]:
dataset.set_format("pandas")
df = dataset[:]
df.head(2)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/pull/2955,Update legacy Python image for CI tests in Linux,[],"Instead of legacy, use next-generation conveni..."
1,https://github.com/huggingface/datasets/pull/2954,Run tests in parallel,[There is a speed up in Windows machines:\r\n-...,Run CI tests in parallel to speed up the test ...


In [ ]:
df = df.explode('comments',ignore_index=True)
df.head(1)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/pull/2955,Update legacy Python image for CI tests in Linux,NaN,"Instead of legacy, use next-generation conveni..."


In [ ]:
df.isna().sum()

html_url       0
title          0
comments    1177
body          20
dtype: int64

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
from datasets import Dataset
data_e = Dataset.from_pandas(df)
data_e

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 6049
})

In [ ]:
data_e = data_e.map(lambda x:{'comment_len':len(x['comments'])})

Map:   0%|          | 0/6049 [00:00<?, ? examples/s]

In [ ]:
data_e = data_e.filter(lambda x:x['comment_len']>15)

Filter:   0%|          | 0/6049 [00:00<?, ? examples/s]

In [ ]:
data_e = data_e.map(lambda x:{'text': x['title'] + '\n' + x['comments'] + '\n' + x['body']})


Map:   0%|          | 0/5868 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModel,AutoTokenizer
import torch
model = AutoModel.from_pretrained('sentence-transformers/multi-qa-mpnet-base-dot-v1')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/multi-qa-mpnet-base-dot-v1')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [ ]:
input_ids = tokenizer('this is sentence one',return_tensors='pt',padding=True, truncation=True)
out = model(**input_ids)

In [ ]:
out.last_hidden_state.size(),out.last_hidden_state[:,0,:].size()

(torch.Size([1, 6, 768]), torch.Size([1, 768]))

In [ ]:
def cls_polling(model_out):
    return model_out.last_hidden_state[:,0,:]

In [ ]:
def get_embedding(text):
    input_ids = tokenizer(text,return_tensors='pt',padding=True, truncation=True)
    input_ids.to(device)
    out = model(**input_ids)
    embedding = cls_polling(out)
    return embedding

In [ ]:
get_embedding("This is sentence One").size()

torch.Size([1, 768])

In [ ]:
data_sample = data_e.select(range(10))

In [ ]:
data_sample = data_sample.map(lambda x:{'embedding':get_embedding(x['text']).detach().cpu().numpy()[0]})

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
!pip install faiss-cpu
# pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 18.7 MB/s eta 0:00:00


In [ ]:
data_sample.add_faiss_index(column="embedding")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_len', 'text', 'embedding'],
    num_rows: 10
})

In [ ]:
question = "How can I load a dataset offline?"
question_embedding = get_embedding([question]).cpu().detach().numpy()


In [ ]:
scores, samples = data_sample.get_nearest_examples('embedding',question_embedding, k=5)

In [ ]:
scores

array([51.138054, 51.804695, 54.227325, 55.415665, 57.1123  ],
      dtype=float32)